In [1]:
import redis
from redis import ConnectionPool
import numpy as np
import pickle
from collections import defaultdict
pool1 = ConnectionPool(host='localhost', port=6379, db=1)
r1 = redis.Redis(connection_pool=pool1)
pool4 = ConnectionPool(host='localhost', port=6379, db=4)
r4 = redis.Redis(connection_pool=pool4)
pool2 = ConnectionPool(host='localhost', port=6379, db=2)
r2 = redis.Redis(connection_pool=pool2)
pool3 = ConnectionPool(host='localhost', port=6379, db=3)
r3 = redis.Redis(connection_pool=pool3)


In [2]:
#===========테스트용============ㄱ
ga_s= r4.keys()


In [3]:
ga = ga_s[14].decode()
print(ga)
#===============================ㄴ

2e1c5c315aa6f4d2abfc18b8a0ca7e298e907dd8714a31243e726d08da2f802e


In [7]:
# ga='123'


## 개인 열람 기록
seed_4 = r4.get(ga)
if seed_4 ==None: # 기사를 본적이 없는 ga
    print('return None') # '없음' 신호 넘김
    # return "None"
    # break

#

dics4 = pickle.loads(seed_4)  # 기사 열람기록 + vector
# print(dics4['vecs'])
vec_len = len(dics4['vecs'])
print(vec_len)

history = r3.get(ga)   # 추천기록
history = pickle.loads(history) if history!=None else defaultdict(int)
out_list = np.array([k for k, v in history.items() if v >= 3])
# reversep = history['read']

## 전체 mat
mat = r2.get('mat')
mat = np.frombuffer(mat, dtype='float32').reshape(-1, 50).copy()
gid_list = np.frombuffer(r2.get('gid'), dtype='U9')
title_list = np.frombuffer(r2.get('title'), dtype='U59')
url_list = np.frombuffer(r2.get('url'), dtype='U68')
thumburl_list = np.frombuffer(r2.get('thumburl'), dtype='U65')

### de_index 설정
del_index = np.where(np.isin(gid_list, out_list))
mat[del_index] =0

4


In [14]:
if vec_len == 1:
    vec = dics4['vecs'][0]
    sorted_g = np.argsort(np.matmul(mat, vec))[::-1]
    first0 = sorted_g[:9]
    last0 = sorted_g[-5:]
    pallet = [[first0, 9], [last0,3]]

elif vec_len ==2:
    first_vec = dics4['vecs'][-1]  # 멘 뒤에있는게 최신
    last_vec = dics4['vecs'][0]
    first0 = np.argsort(np.matmul(mat, first_vec))[::-1][:6]
    mat[first0] = 0
    sorted_last = np.argsort(np.matmul(mat, last_vec))[::-1]
    last0 = sorted_last[:12]
    pallet = [[first0, 6], [last0,6]]
else:
    print("여긴가")
    indiv_mat = np.array(dics4['vecs'])
    first_vec = indiv_mat[-1].copy()
    ordered0 = np.argsort(np.matmul(indiv_mat, first_vec))[::-1]
    middle_vec = indiv_mat[ordered0[len(ordered0)//2]]
    last_vec = indiv_mat[ordered0[-1]]

    first0 = np.argsort(np.matmul(mat, first_vec))[::-1][:6]
    mat[first0] = 0
    middle0 = np.argsort(np.matmul(mat, middle_vec))[::-1][:6]
    mat[middle0] = 0
    sorted_last = np.argsort(np.matmul(mat, last_vec))[::-1]
    last0 = sorted_last[:6]
    pallet = [[first0,4], [middle0,4], [last0,4]]




여긴가


In [15]:
top12 = []
for list0, limit0 in pallet:
    temp_n = 0
    for ind0 in list0:
        if ind0 not in top12:
            top12.append(ind0)
            temp_n +=1
            if temp_n >=limit0:
                break
dics1 = {}
for i, x in enumerate(top12):
    dics1[i] = {'title': title_list[x], 'url': url_list[x], 'thumburl': thumburl_list[x]}

    ### history update
    gid0 = gid_list[x]
    history[gid0] += 0.5
    r3.set(ga, pickle.dumps(history))
    r3.expire(ga, 600)
print(dics1)
# return dics1
print('return dics1')

{0: {'title': '[광화문에서/유성열]유권자들의 마음을 다른 걸로 잡을 순 없나', 'url': 'https://www.donga.com/news/Column/article/all/20240118/123124675/1', 'thumburl': 'https://dimg.donga.com/wps/NEWS/IMAGE/2024/01/18/123124673.1.jpg'}, 1: {'title': '김종인 “신당 세력들 각자도생 안 돼…단일대오로 총선 맞이하길”', 'url': 'https://www.donga.com/news/Politics/article/all/20240120/123145637/2', 'thumburl': 'https://dimg.donga.com/wps/NEWS/IMAGE/2024/01/20/123146224.1.jpg'}, 2: {'title': '민주 탈당-공천 잡음에 ‘지지율 정체’…이재명 공천 관리 ‘난제’', 'url': 'https://www.donga.com/news/Politics/article/all/20240121/123150112/1', 'thumburl': 'https://dimg.donga.com/wps/NEWS/IMAGE/2024/01/21/123150035.2.jpg'}, 3: {'title': '류호정 “‘비례연합정당’ 민주당 주도 아니라는 건 손바닥으로 하늘 가리는 소리”[중립기어 라이브]', 'url': 'https://www.donga.com/news/Politics/article/all/20240118/123122122/1', 'thumburl': 'https://dimg.donga.com/wps/NEWS/IMAGE/2024/01/18/123122114.1.png'}, 4: {'title': '‘이태원 참사 재판’ 가는 김광호…‘세월호’ 못지않은 법리 다툼 예고', 'url': 'https://www.donga.com/news/Society/article/all/20240119/123136859/1', '

In [ ]:
#===================

In [17]:
a=[1,2,3,4,5,6,7]
n = len(a)//2
print(n)
print(a[n])

3
4


In [2]:
ga_s= r4.keys()

In [4]:
for ga in ga_s:
    if not (type(pickle.loads(r4.get(ga))['vecs'])==list):
        print(ga)

In [22]:
r4.delete('0d461b1c768d443e6488a37292b4f84ac9bb0de9ef5a8e0b79068f023327ea4a')

1